In [7]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, GRU
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
import matplotlib.pyplot as plt
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam
import time
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf


# Cargar los datos
data = pd.read_csv("../TransferLearning/Cluster0ReadyToNN.csv", sep=";")

In [8]:
# Crear un diccionario para almacenar los objetos scaler por grupo
scalers = {}

# Iterar sobre los grupos únicos en Column15
for group in data['Column15'].unique():
    # Filtrar datos por grupo
    group_data = data[data['Column15'] == group]

    # Seleccionar las columnas para normalización (las 13 primeras)
    features = group_data.iloc[:, :13]

    # Normalizar los datos con MinMaxScaler
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(features)

    # Almacenar el scaler en el diccionario
    scalers[group] = scaler

    # Actualizar el DataFrame con los datos normalizados
    data.loc[data['Column15'] == group, 'Column1':'Column13'] = normalized_data

In [9]:
# Ordenar el DataFrame por 'Column 14' de forma ascendente
data = data.sort_values(by='Column14')

# Dividir los datos en entrenamiento (70%) y temporal (30%)
train_temp_data, test_data = train_test_split(data, test_size=0.3, stratify=data['Column15'], random_state=0)
#train_temp_data, test_data = train_test_split(data, test_size=0.3, shuffle=False, random_state=0)

# Dividir el temporal en entrenamiento (70%) y validación (30%)
train_data, validation_data = train_test_split(train_temp_data, test_size=0.3, stratify=train_temp_data['Column15'], random_state=0)
#train_data, validation_data = train_test_split(train_temp_data, test_size=0.3, shuffle=False, random_state=0)

# Separar características (X) y columna objetivo (y)
X_train = train_data.iloc[:, :12]
y_train = train_data['Column13']
X_val = validation_data.iloc[:, :12]
y_val = validation_data['Column13']
X_test = test_data.iloc[:, :12]
y_test = test_data['Column13']


# Reshape de los datos para GRU (número de muestras, número de pasos de tiempo, número de características)
n_samples_train, n_features = X_train.shape
n_samples_val = X_val.shape[0]
n_timesteps = 1
X_train = X_train.values.reshape(n_samples_train, n_timesteps, n_features)
X_val = X_val.values.reshape(n_samples_val, n_timesteps, n_features)
X_test = X_test.values.reshape(X_test.shape[0], n_timesteps, n_features)

print(n_samples_train, n_samples_val, n_features)

83587 35823 12


In [10]:
# Definir la función para construir el modelo
def build_model(hp):
    model = Sequential()

    # Primera capa GRU con input_shape y 12 neuronas fijas
    model.add(GRU(
        units=12,  # Fijo en 12 neuronas
        activation='tanh',
        input_shape=(n_timesteps, n_features),  # Solo en la primera capa
        return_sequences=True  # Para permitir capas ocultas adicionales
    ))
    
    # Número de capas ocultas
    num_layers = hp.Int('num_layers', 5, 15)
    
    # Capas ocultas GRU con número de neuronas optimizado
    for i in range(num_layers):
        model.add(GRU(
            units=hp.Int(f'units_{i}', min_value=20, max_value=80, step=10),
            activation='tanh',
            return_sequences=(i < num_layers - 1)  # Solo True en capas intermedias
        ))

    # Capa de salida con 1 neurona
    model.add(Dense(1, activation='tanh'))

    # Optimizador y tasa de aprendizaje aleatoria
    optimizer = Adam(
    learning_rate=hp.Float('learning_rate', min_value=0.00001, max_value=0.001, sampling='log')
    )
    
    model.compile(
        loss='mean_absolute_error',
        optimizer=optimizer,
        metrics=['mean_absolute_error']
    )
    
    return model

In [14]:
#Params
epochs_val = 100
batch_size_val = 256
max_trials_val = 15

In [15]:
# Definir el hiperbuscador RandomSearch
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=max_trials_val,  # Número de iteraciones
    directory='my_tuner_dir',
    project_name='GRU_optimization_norm'
)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, min_delta=1E-4)


# Ejecutar la búsqueda de hiperparámetros
tuner.search(X_train, y_train, epochs=epochs_val, batch_size=batch_size_val, validation_data=(X_val, y_val), callbacks=[callback])

Trial 15 Complete [00h 17m 48s]
val_loss: 0.0929826945066452

Best val_loss So Far: 0.08473043888807297
Total elapsed time: 03h 19m 44s


In [17]:
# Obtener el mejor modelo
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.get_best_models(num_models=1)[0]

# Predicciones con el mejor modelo
y_pred = best_model.predict(X_test)

# Mostrar los mejores valores encontrados
print("Mejores hiperparámetros encontrados:")
print(f" - Número de capas GRU: {best_hps.get('num_layers')}")
for i in range(best_hps.get('num_layers') - 1):  # -1 porque la primera capa es fija
    print(f" - Unidades en GRU {i + 1}: {best_hps.get(f'units_{i}')}")
print(f" - Tasa de aprendizaje: {best_hps.get('learning_rate')}")
#print(f" - Optimizador: {best_hps.get('optimizer')}")

1600/1600 [==============================] - 6s 2ms/step
Mejores hiperparámetros encontrados:
 - Número de capas GRU: 6
 - Unidades en GRU 1: 30
 - Unidades en GRU 2: 20
 - Unidades en GRU 3: 80
 - Unidades en GRU 4: 50
 - Unidades en GRU 5: 20
 - Tasa de aprendizaje: 0.000552474049906462


In [26]:
# Fine-tuning del mejor modelo
start_time = time.time()
history = best_model.fit(X_train, y_train, epochs=epochs_val, batch_size=batch_size_val, validation_data=(X_val, y_val), callbacks=[callback])
end_time = time.time()

# Calcular el tiempo total
training_time = end_time - start_time
print(f"Tiempo de entrenamiento del mejor modelo: {training_time:.2f} segundos")

# Predicciones con el mejor modelo
y_pred = best_model.predict(X_test)

Epoch 1/100
327/327 [==============================] - 4s 12ms/step - loss: 0.0848 - mean_absolute_error: 0.0848 - val_loss: 0.0849 - val_mean_absolute_error: 0.0849
Epoch 2/100
327/327 [==============================] - 4s 12ms/step - loss: 0.0847 - mean_absolute_error: 0.0847 - val_loss: 0.0848 - val_mean_absolute_error: 0.0848
Epoch 3/100
327/327 [==============================] - 4s 11ms/step - loss: 0.0847 - mean_absolute_error: 0.0847 - val_loss: 0.0847 - val_mean_absolute_error: 0.0847
Epoch 4/100
327/327 [==============================] - 4s 12ms/step - loss: 0.0847 - mean_absolute_error: 0.0847 - val_loss: 0.0848 - val_mean_absolute_error: 0.0848
Epoch 5/100
327/327 [==============================] - 4s 12ms/step - loss: 0.0848 - mean_absolute_error: 0.0848 - val_loss: 0.0848 - val_mean_absolute_error: 0.0848
Epoch 6/100
327/327 [==============================] - 4s 11ms/step - loss: 0.0848 - mean_absolute_error: 0.0848 - val_loss: 0.0846 - val_mean_absolute_error: 0.0846
Epoc

In [27]:
# Crear un DataFrame con las predicciones desnormalizadas y los valores reales
resultados = pd.DataFrame({'Valor Real': y_test.values.flatten(), 'Predicciones': y_pred.flatten()})
print(resultados)

# Agregar la columna de predicciones al conjunto de prueba
test_data['Predicted_Column13'] = y_pred.flatten()

# Crear un DataFrame para almacenar los resultados desnormalizados
desnormalized_test_data = test_data.copy()

# Desnormalizar 'Column1' a 'Column13' y 'Predicted_Column13' según la normalización por grupos
for group, scalerY in scalers.items():
    # Filtrar el conjunto de prueba correspondiente al grupo
    group_test_data = test_data[test_data['Column15'] == group]

    # Seleccionar las columnas normalizadas para desnormalizar
    normalized_features = group_test_data[['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6', 'Column7', 'Column8', 'Column9', 'Column10', 'Column11', 'Column13', 'Predicted_Column13']]

    # Desnormalizar los datos utilizando el objeto scalerY correspondiente
    original_data = scalerY.inverse_transform(normalized_features)

    # Crear un DataFrame temporal para almacenar los datos desnormalizados
    temp_df = pd.DataFrame(original_data, columns=['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6', 'Column7', 'Column8', 'Column9', 'Column10', 'Column11', 'Column13', 'Predicted_Column13'])

    # Actualizar el DataFrame desnormalizado con los datos desnormalizados
    desnormalized_test_data.loc[desnormalized_test_data['Column15'] == group, ['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6', 'Column7', 'Column8', 'Column9', 'Column10', 'Column11', 'Column13', 'Predicted_Column13']] = temp_df.values

# Imprimir el conjunto de prueba después de la desnormalización
print(desnormalized_test_data)

# Eliminar todas las columnas excepto las últimas cuatro
resultados = desnormalized_test_data.iloc[:, -4:]

# Imprimir el conjunto de prueba después de la eliminación de columnas
print(resultados)

       Valor Real  Predicciones
0        0.128378      0.218927
1        0.347826      0.263083
2        0.051724      0.109681
3        0.295400      0.214549
4        0.155556      0.074272
...           ...           ...
51171    0.321429      0.126699
51172    0.011936      0.039805
51173    0.053030      0.076405
51174    0.312155      0.210788
51175    0.600000      0.433262

[51176 rows x 2 columns]


C:\Users\AdrianGilGamboa\AppData\Local\Temp\ipykernel_15844\619335387.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[18.57464267 20.17045368 21.89338557 18.71620618 18.76817335 19.65853965
 19.71085958 18.64047307 42.39401025 19.10411804 45.2288695  19.4616177
 44.26087672 16.81710053 17.52766469 19.10879684 22.31652966]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  desnormalized_test_data.loc[desnormalized_test_data['Column15'] == group, ['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6', 'Column7', 'Column8', 'Column9', 'Column10', 'Column11', 'Column13', 'Predicted_Column13']] = temp_df.values


        Column1  Column2  Column3  Column4  Column5  Column6  Column7  \
24905      58.0     66.0     62.0     56.0     77.0     60.0    170.0   
155320    101.0     39.0     24.0     28.0     65.0     32.0     28.0   
8745       30.0     30.0     29.0     31.0     31.0     31.0     29.0   
60462     193.0     59.0     48.0     87.0    186.0     59.0     47.0   
4241       35.0     29.0     37.0     31.0     29.0     32.0     32.0   
...         ...      ...      ...      ...      ...      ...      ...   
132964     37.0     35.0     33.0     35.0     34.0     36.0     21.0   
130033      5.0      8.0      7.0     11.0      1.0      7.0      8.0   
124375    137.0    120.0     63.0     60.0     54.0     73.0      6.0   
50855     185.0     32.0     49.0     67.0     51.0     37.0     56.0   
5442       29.0     65.0     44.0     31.0     50.0     69.0     48.0   

        Column8  Column9  Column10  Column11  Column12  Column13 Column14  \
24905      69.0     66.0      48.0      53.0  

In [28]:
# Obtener y_test_norm de la primera columna de resultados
y_test = resultados['Column13']

# Obtener y_pred de la última columna del conjunto de prueba después de la desnormalización
y_pred = desnormalized_test_data['Predicted_Column13']

# Imprimir y_test_norm y y_pred
print("y_test:", y_test)
print("y_pred:", y_pred)


# Imprimir el DataFrame
print(resultados)

# Calcular RMSE con datos desnormalizados
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'RMSE en el conjunto de prueba: {rmse}')

# Calcular MAE con datos desnormalizados
mae = mean_absolute_error(y_test, y_pred)
print(f'MAE en el conjunto de prueba: {mae}')

# Calcular MRE con datos desnormalizados
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'MAPE en el conjunto de prueba: {mape}')

# Calcula la duración del entrenamiento en segundos
training_duration = end_time - start_time
# Imprime el tiempo de entrenamiento en segundos y en formato de horas, minutos y segundos
print(f'Tiempo de entrenamiento: {training_duration:.2f} segundos')
#print(f'Tiempo de entrenamiento (HH:MM:SS): {int(training_duration // 3600)}:{int((training_duration % 3600) // 60)}:{int(training_duration % 60)}')

y_test: 24905      41.0
155320     59.0
8745       23.0
60462     150.0
4241       34.0
          ...  
132964     65.5
130033     10.0
124375     13.0
50855     145.0
5442       85.0
Name: Column13, Length: 51176, dtype: float64
y_pred: 24905      54.401257
155320     45.356331
8745       26.361490
60462     116.608665
4241       23.026784
             ...    
132964     19.547575
130033     31.012969
124375     16.085454
50855     108.305314
5442       66.658822
Name: Predicted_Column13, Length: 51176, dtype: float64
        Column13 Column14  Column15  Predicted_Column13
24905       41.0   2005/2      8751           54.401257
155320      59.0   2006/3     56704           45.356331
8745        23.0   2015/2      3059           26.361490
60462      150.0   2011/3     20496          116.608665
4241        34.0   2009/4      1242           23.026784
...          ...      ...       ...                 ...
132964      65.5   2006/3     46569           19.547575
130033      10.0   2002/4  

In [31]:
# Guardar el DataFrame en un archivo CSV
resultados.to_csv("GRU_OPT_FineTuning_Norm(1-15iter).csv", index=False)

In [32]:
# Convertir la columna 'Column14' para extraer el año
resultados['Año'] = resultados['Column14'].str.split('/').str[0].astype(int)

# Agrupar por año y calcular la suma de reales y predicciones
suma_anual = resultados.groupby('Año').agg({
    'Column13': 'sum',  # Suma de valores reales
    'Predicted_Column13': 'sum'  # Suma de predicciones
}).reset_index()

# Agregar una columna de MAPE por fila
suma_anual['MAPE_fila'] = (
    (abs(suma_anual['Column13'] - suma_anual['Predicted_Column13']) / suma_anual['Column13']) * 100
)

# Calcular la media de la columna MAPE
media_mape = suma_anual['MAPE_fila'].mean()

# Mostrar el DataFrame actualizado y la media
print(suma_anual)
print(f"Media de MAPE (por fila): {media_mape:.2f}%")

     Año       Column13  Predicted_Column13  MAPE_fila
0   2002  104265.678499        90364.398985  13.332556
1   2003  177491.021691       142248.354296  19.856028
2   2004  164849.403889       141290.380755  14.291240
3   2005  159059.300841       135206.596639  14.996108
4   2006  131381.875214       119955.431387   8.697123
5   2007  131223.654627       113875.387802  13.220381
6   2008  122996.287059       110850.921703   9.874579
7   2009  128977.774184       106363.161978  17.533728
8   2010  117483.995234       101577.369558  13.539398
9   2011  118736.260722       106036.447701  10.695817
10  2012  122697.561296       109830.503874  10.486808
11  2013  109818.900063       100818.243132   8.195909
12  2014  104937.636563        97517.198232   7.071284
13  2015  109243.945117        98968.646991   9.405829
Media de MAPE (por fila): 12.23%
